In [7]:
import pandas as pd
from sqlalchemy import create_engine

def establish_connection():
    # Establish a connection to the Sakila database
    engine = create_engine("mysql+pymysql://root:4p;H;01:3~'n@localhost/sakila")
    return engine

def rentals_month(engine, month, year):
    # Execute SQL query to retrieve rental data for the specified month and year
    query = f"""
            SELECT 
                customer_id,
                COUNT(rental_id) AS rentals_{month}_{year}
            FROM 
                rental
            WHERE 
                MONTH(rental_date) = {month} AND 
                YEAR(rental_date) = {year}
            GROUP BY 
                customer_id
            """
    # Fetch the results as a Pandas DataFrame
    with engine.connect() as conn:
        rental_data = pd.read_sql_query(query, conn)
    return rental_data

def rental_count_month(rental_data, month, year):
    # Rename the rentals column to match the month and year
    rental_data.rename(columns={'rental_count': f'rentals_{month}_{year}'}, inplace=True)
    return rental_data

def compare_rentals(rental_data_month1, rental_data_month2, month1, year1, month2, year2):
    # Merge the two DataFrames based on customer_id
    combined_rentals = pd.merge(rental_data_month1, rental_data_month2, on='customer_id', how='outer')
    # Calculate the difference between rentals for the two months
    combined_rentals['difference'] = combined_rentals[f'rentals_{month1}_{year1}'] - combined_rentals[f'rentals_{month2}_{year2}']
    return combined_rentals
